In [163]:
# importa as bibliotecas necessárias
import PyPDF2
import re
import pandas as pd
import os

In [164]:

path = r'C:\Users\MatheusPereira\OneDrive - Pontte\Área de Trabalho\automacaoRegistroCCI\Contratos\FI_Contrato_RNContabil_Assinatura Digital_172464-Manifesto1.pdf'


In [165]:
# Teste de criação de lista de arquivos

files_dir = r"C:\Users\MatheusPereira\OneDrive - Pontte\Área de Trabalho\automacaoRegistroCCI\Contratos"
contratoFiles = [f for f in os.listdir(files_dir) if f.endswith(".pdf")]
contratoFiles

['FI_Contrato_Juliana_Assinatura Digital.pdf',
 'FI_Contrato_RNContabil_Assinatura Digital_172464-Manifesto1.pdf',
 'FI_Contrato_Romilton_Assinatura Digital-Manifesto.pdf',
 'HE_Contrato_Agostinho_Assinatura Digital_VFinal.pdf',
 'HE_Contrato_Conrado_Assinatura Digital_VFinal - Conrado.pdf',
 'HE_Contrato_Margareide_Assinatura Digital_VFinal.pdf',
 'HE_Contrato_Oliboni _Assinatura Digital_VFinal.pdf',
 'HE_Contrato_PatriciaMarcondes_Assinatura Digital_VFinal.pdf']

In [166]:
# Abre o arquivo pdf 
i = 1
pdf_file = os.path.join(files_dir, contratoFiles[i])


In [167]:
#Faz a leitura usando a biblioteca
read_pdf = PyPDF2.PdfFileReader(pdf_file)

# pega o numero de páginas
number_of_pages = read_pdf.getNumPages()

#Extriar Texto Página 1 a 5
text=''
for i in range(0,6):
    #Ler Página PDF
    pageObj = read_pdf.getPage(i)
    #Extrair Texto
    text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)
text = re.sub(' {2,}', ' ', text).strip(' ')
text = re.sub(' :', ':', text)

In [168]:
text

"INSTRUMENTO PARTICULAR DE COMPRA E VENDA DE IMÓVEL, FINANCIAMENTO COM PACTO ADJETO DE ALIENAÇÃO FIDUCIÁRIA EM GARANTI A, EMISSÃO DE CÉDULA DE CRÉDITO IMOBILIÁRIO – CCI E OUTRAS AVENÇAS Nº 2205055482/RNR Esta página é parte integrante e inseparável do Instrumento Particular de Compra e Ven da de Imóvel, Financiamento com Pacto Adjeto de Alienação Fiduciária em Garantia, Emissão de Cédula de Crédito Imobiliário – CCI e Outras Avenças nº 2205055482/RNR CAMPO 1 – CREDORA RAZÃO SOCIAL: QI SOCIEDADE DE CRÉDITO DIRETO S.A. ENDEREÇO COMERCIAL: Avenida Brigadeiro Faria Lima, nº 2.391, 1º andar COMPLEMENTO: Conjunto 12 BAIRRO: Jardim Paulistano CIDADE: São Paulo UF: SP CEP: 01452-000 CNPJ: 32.402.502/0001-35 DATA DA CONSTITUIÇÃO: 10/01/2019 REGISTRO JUCESP: 35300530225 REPRESENTANTES: Roberto Amaral Dantas Junior, brasileiro, portador da cédula de identidade RG nº 50.016.943-3 SSP/SP e inscrito no CPF sob nº 400.516.238-03, nos termos da Procuração Pública lavrada pelo 1º Tabelião de Notas de S

In [169]:
listaDePara = {'valorTotal':'1. Valor do Financiamento: R$','tabela': 'Sistema de Amortização:','Taxa ao Mes':'2.1.1. Juros de','registro':'4.3. Despesas','valorLiquido': '7. Valor Líquido a Liberar do Financiamento: R$',
                'prazoMes': 'PRAZO DE AMORTIZAÇÃO:','valorPrimeiraParcelaComEncargos':'VALOR TOTAL DO PRIMEIRO ENCARGO, NESTA DATA: R$',
                'valorImóvel':'Imóvel para fins de leilão: R$','prazoContrato': 'N.º DE PRESTAÇÕES:','ultimaParcela':'DATA DE VENCIMENTO DA ÚLTIMA PRESTAÇÃO:','dataContrato': 'Data de Liberação dos Recursos: ',
                'valorPrimeiraParcela':'), NESTA DATA: R$','primeiraParcela':'DATA DE VENCIMENTO DA PRIMEIRA PRESTAÇÃO: ','indice':'ÍNDICE: MENSAL do'
                }

listaKey = []
listaValues = []

for key, value in listaDePara.items():
    if key == 'registro':

        inicFrase = '4.3. Despesas'
        fimFrase = 'Registro: R$'

        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(inicFrase, 0)
        finalTopico = text.find(fimFrase, 0)

        #Criar Paragráfo Auxiliar (Somente com os sub itens do tópico 4. Despesas)
        paragrafo4 = text[inicioTopico+len(inicFrase)+1:finalTopico-1]
        value = f'{inicFrase} {paragrafo4} {fimFrase}'
        
        

  
    inicioFrase = text.find(value,0)
    finalFrase = inicioFrase + len(value) + 1
    proximoEspaco = text.find(" ", finalFrase)
    valorExtraido = text[finalFrase:proximoEspaco]

    #Ajustar Valores Númericos
    if '.' in valorExtraido:
        valorExtraido = valorExtraido.replace(".", "")
        valorExtraido = valorExtraido.replace(",", ".")
    
    #Ajustar Valores Percentuais
    if '%' in valorExtraido:
        valorExtraido = valorExtraido.replace(",", ".")
        valorExtraido = valorExtraido.replace("%", "")
        valorExtraido = round(float(valorExtraido)/100,4)
    elif 'IPCA' or 'IGPM' in valorExtraido:
        valorExtraido = valorExtraido.replace(",", "")
    
    elif key == 'registro':
        #Definir Variáveis Auxiliares
        topico4 = '4. Despesas'
        topico5 = '5. Valor Destinado'

        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(topico4, 0)
        finalTopico = text.find(topico5, 0)

        #Criar Paragráfo Auxiliar (Somente com os sub itens do tópico 4. Despesas)
        paragrafo4 = text[inicioTopico+len(topico4)+1:finalTopico-1]
        paragrafo4 = re.sub('\s+',' ', paragrafo4)

        listaChave = ['4.3.','4.4.']
        inicioItens = []

        for item in listaChave:
            inicioP = paragrafo4.find(item,0)
            inicioItens.append(inicioP)
            
        item1 = paragrafo4[inicioItens[0]:inicioItens[1]-1]

        listaFinal = [item1]
        print(listaFinal)
        listaValor = []

        for itemAux in listaFinal:
            if '[X]' in itemAux:
                inicioAux = itemAux.find('R$ ', 0)
                fimAux = itemAux.find(",", inicioAux) + 3
                valorExtraido = itemAux[inicioAux+3:fimAux]
            else:
                valorExtraido = '0,00'
    listaKey.append(key)
    listaValues.append(valorExtraido)


In [170]:
# Extraindo numero da matricula
inicioFrase = text.find('Matriculado sob nº ',0)
finalFrase = inicioFrase + len('Matriculado sob nº') + 1
ultimaMatricula = text.find("no", finalFrase)
valorExtraido = text[finalFrase:ultimaMatricula]
valorExtraido = valorExtraido.replace(",", "")
valorExtraido = valorExtraido.replace(" ", "")

listaKey.append('Matrícula')
listaValues.append(valorExtraido)
valorExtraido

'172.464'

In [171]:
# Extraindo cartório
inicioFrase = text.find(f'Matriculado sob nº {valorExtraido}, no',0)
finalFrase = inicioFrase + len(f'Matriculado sob nº {valorExtraido}, no') + 1
ultimaMatricula = text.find("Forma de aquisição", finalFrase)
valorExtraido = text[finalFrase:ultimaMatricula-2]

listaKey.append('Cartório')
listaValues.append(valorExtraido)
valorExtraido

'Oficial de Registro de Imóveis de Barueri/S'

In [172]:
# Extraindo participantes da operação 
campo5 = 'fins de indenização do Seguro'                         #inicio e fim da extração
campo6 = 'VI – CLÁUSULA(S)'

#Pegar posição das variáveis auxiliares no texto
inicioTopico = text.find(campo5, 0)
finalTopico = text.find(campo6, 0)

#Criar Paragráfo Auxiliar
paragrafoAux = 0
paragrafoAux = text[inicioTopico+len(campo5)+1:finalTopico-1]
paragrafoAux = re.sub('\s+',' ', paragrafoAux)
totalParticipantes = paragrafoAux.count('Nome: ')

paragrafoAux

"PRA E VENDA DE IMÓVEL, FINANCIAMENTO COM PACTO ADJETO DE ALIENAÇÃO FIDUCIÁRIA EM GARANTI A, EMISSÃO DE CÉDULA DE CRÉDITO IMOBILIÁRIO – CCI E OUTRAS AVENÇAS Nº 2205055482/RNR Esta página é parte integrante e inseparável do Instrumento Particular de Compra e Ven da de Imóvel, Financiamento com Pacto Adjeto de Alienação Fiduciária em Garantia, Emissão de Cédula de Crédito Imobiliário – CCI e Outras Avenças nº 2205055482/RNR CAMPO 1 – CREDORA RAZÃO SOCIAL: QI SOCIEDADE DE CRÉDITO DIRETO S.A. ENDEREÇO COMERCIAL: Avenida Brigadeiro Faria Lima, nº 2.391, 1º andar COMPLEMENTO: Conjunto 12 BAIRRO: Jardim Paulistano CIDADE: São Paulo UF: SP CEP: 01452-000 CNPJ: 32.402.502/0001-35 DATA DA CONSTITUIÇÃO: 10/01/2019 REGISTRO JUCESP: 35300530225 REPRESENTANTES: Roberto Amaral Dantas Junior, brasileiro, portador da cédula de identidade RG nº 50.016.943-3 SSP/SP e inscrito no CPF sob nº 400.516.238-03, nos termos da Procuração Pública lavrada pelo 1º Tabelião de Notas de São Paulo/SP no livro 4831, pá

In [173]:
participantesKey = []
participantesValues = []

# Extraindo participantes
fimValor = 0

for num in range(0, totalParticipantes):
    paragrafoAux = paragrafoAux[fimValor:]
    inicioFrase = paragrafoAux.find('Nome: ',0)
    finalFrase = inicioFrase + len('Nome: ') 
    ultimoNome = paragrafoAux.find("Participação: ", finalFrase)
    nomeExtraido = paragrafoAux[finalFrase:ultimoNome]
    participantesKey.append(f'Participante{num+1}')
    participantesKey.append(f'Participação{num+1}')
    participantesValues.append(nomeExtraido.strip())
    inicioFrase = paragrafoAux.find('Participação:',0)
    finalFrase = inicioFrase + len('Participação:') 
    fimValor = paragrafoAux.find("%", finalFrase)
    participacaoExtraido = paragrafoAux[finalFrase:fimValor+1]
    participantesValues.append(participacaoExtraido.strip())
    dict_participantes = dict(zip(participantesKey,participantesValues))
    print(dict_participantes)

{'Participante1': 'RENAN DA SILVA RODRIGUES', 'Participação1': '100%'}


In [174]:
#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))
dict_keyValue.update(dict_participantes)
dict_keyValue

{'valorTotal': '512304.23',
 'tabela': 'SAC',
 'Taxa ao Mes': 0.007,
 'registro': '15000.00',
 'valorLiquido': '449500.00',
 'prazoMes': '360',
 'valorPrimeiraParcelaComEncargos': '5332.43',
 'valorImóvel': '758500.00',
 'prazoContrato': '357',
 'ultimaParcela': '15/05/2052',
 'dataContrato': '5/05/2022',
 'valorPrimeiraParcela': '5144.42',
 'primeiraParcela': '5/09/2022',
 'indice': 'IPCA',
 'Matrícula': '172.464',
 'Cartório': 'Oficial de Registro de Imóveis de Barueri/S',
 'Participante1': 'RENAN DA SILVA RODRIGUES',
 'Participação1': '100%'}